In [1]:
from whitespace_correction import WhitespaceCorrector
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModel
import os
from sentence_transformers.util import cos_sim
import pandas as pd

os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [2]:
# model_semantic_similarity = AutoModel.from_pretrained('/home/css/models/NV-Embed-v2', trust_remote_code=True, device_map="auto")

# def calculate_semantic_similarity(sentence1, sentence2, max_length=32768):

#     task_name_to_instruct = {"STS": "Retrieve semantically similar text."} # STS: 语义相似度任务
#     query_prefix = "Instruct: "+task_name_to_instruct["STS"]+"\nQuery: "
#     # 对输入的两个句子进行编码
#     claims_embeds = model_semantic_similarity.encode([
#         sentence1
#     ]
#     , instruction=query_prefix, max_length=32768)

#     contexts_embeds = model_semantic_similarity.encode([
#             sentence2
#         ]
#     , instruction=query_prefix, max_length=32768)
    
#     # 计算余弦相似度
#     similarity = cos_sim(claims_embeds, contexts_embeds)
    
#     return similarity.item()

In [3]:
model_semantic_similarity = AutoModel.from_pretrained('/home/css/models/NV-Embed-v2', 
                                                      trust_remote_code=True, 
                                                      device_map="auto",
                                                      torch_dtype='bfloat16',
                                                      )

def calculate_semantic_similarity(sentence1, sentence2, max_length=32768):
    # 对输入的两个句子进行编码
    embeddings = model_semantic_similarity.encode([sentence1, sentence2], 
                                                  instruction="", 
                                                  max_length=max_length)
    
    # 计算余弦相似度
    similarity = cos_sim(embeddings[0], embeddings[1])
    
    return similarity.item()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [4]:
# text_1="The lecturer went against the authour 's insistence for the following reasons ."
# text_2="The lecturer went against the authour's insistence for the following reasons."
# calculate_semantic_similarity(text_1,text_2)

text_1="the fact of the matter is that a lot of acadmic subjects can not be used separately ."
text_2="the fact of the matter is that a lot of acadmic subjects cannot be used separately."
calculate_semantic_similarity(text_1,text_2)

/home/liujunhui/.cache/huggingface/modules/transformers_modules/NV-Embed-v2/modeling_nvembed.py:349: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(batch_dict.get('input_ids').to(batch_dict.get('input_ids')).long()),
/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


0.9726235866546631

In [4]:
WhitespaceCorrector.available_models()

[ModelInfo(name='eo_large_byte', description='Byte-level model combining fast inference and good quality', tags=['default', 'lang::en', 'arch::encoder-only', 'input::byte']),
 ModelInfo(name='eo_large_char', description='Character-level model combining fast inference and good quality', tags=['lang::en', 'arch::encoder-only', 'input::char']),
 ModelInfo(name='eo_large_char_v1', description='Character-level model combining fast inference and good quality, trained with a different loss than eo_large_char', tags=['lang::en', 'arch::encoder-only', 'input::char']),
 ModelInfo(name='eo_larger_byte', description='Larger and slower than eo_large_byte, but also more accuracte', tags=['lang::en', 'arch::encoder-only', 'input::byte']),
 ModelInfo(name='eo_medium_byte', description='Smaller and faster than eo_large_byte, but less accurate', tags=['lang::en', 'arch::encoder-only', 'input::byte']),
 ModelInfo(name='eo_medium_char', description='Smaller and faster than eo_large_char, but less accurate

In [5]:
cor = WhitespaceCorrector.from_pretrained(model="eo_larger_byte", device="cuda:0", download_dir="/home/css/models/wsc")

2024-11-15 17:31:16,776 [WHITESPACE CORRECTION DOWNLOAD] [INFO] eo_larger_byte is already downloaded to download directory /home/css/models/wsc
/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/text_utils/io.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have

## 1.1读取txt文件，wsc，保存txt文件

In [6]:
# 文本文件路径
tt_lst = [0, 1]
for tt in tt_lst:
    file_name = ["sources.txt", "corrections.txt"]
    input_file_path = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg/" + file_name[tt]
    output_file_path = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg_corrected/" + file_name[tt]

    # 打开文件并读取内容到列表
    with open(input_file_path, "r") as file:
        jfleg_lst = [line.strip() for line in file.readlines()]


    jfleg_lst_output = []
    for item in tqdm(jfleg_lst):
        corrected_string = cor.correct_text(item)
        jfleg_lst_output.append(corrected_string)

    # 将列表内容写入到txt文件中，最后一行不加换行符
    with open(output_file_path, "w") as file:
        for i, line in enumerate(jfleg_lst_output):
            if i < len(jfleg_lst_output) - 1:
                file.write(line + "\n")
            else:
                file.write(line)

  0%|          | 0/1601 [00:00<?, ?it/s]

/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/text_utils/modules/grouping.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=False):


  0%|          | 0/1601 [00:00<?, ?it/s]

## 1.2txt文件 rollback

In [7]:
# 文本文件路径
score = 0.98    #确定阈值，小于阈值的WSC纠正进行回滚
tt_lst = [0, 1]
for tt in tt_lst:
    file_name = ["sources.txt", "corrections.txt"]
    input_file_path = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg/" + file_name[tt]
    output_file_path_rollback = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg_rollback/" + file_name[tt]

    # 打开文件并读取内容到列表
    with open(input_file_path, "r") as file:
        jfleg_lst = [line.strip() for line in file.readlines()]

    jfleg_lst_output_rollback = []
    nn = 0
    for item in tqdm(jfleg_lst):
        ori_text = item
        cor_text = cor.correct_text(item)
        if calculate_semantic_similarity(ori_text, cor_text) < score:
            jfleg_lst_output_rollback.append(ori_text)
            nn += 1
        else:
            jfleg_lst_output_rollback.append(cor_text)

    with open(output_file_path_rollback, "w") as file:
        for i, line in enumerate(jfleg_lst_output_rollback):
            if i < len(jfleg_lst_output_rollback) - 1:
                file.write(line + "\n")
            else:
                file.write(line)

    print(nn)

  0%|          | 0/1601 [00:00<?, ?it/s]

/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/text_utils/modules/grouping.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=False):


772


  0%|          | 0/1601 [00:00<?, ?it/s]

948


## 2.1保存excel文件

In [7]:
# 文本文件路径
score=0.98
tt_lst = [0, 1]
for tt in tt_lst:
    file_name = ["sources", "corrections"]
    input_file_path = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_txt/jfleg/" + file_name[tt] + ".txt"
    output_file_path_1 = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_excel/jfleg/" + file_name[tt] + ".xlsx"
    output_file_path_2 = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_excel/jfleg_corrected/" + file_name[tt] + ".xlsx"
    output_file_path_3 = "/home/liujunhui/workspace/proj/WSC/dataset/jfleg_excel/jfleg_rollback/" + file_name[tt] + ".xlsx"

    # 打开文件并读取内容到列表
    with open(input_file_path, "r") as file:
        jfleg_lst = [line.strip() for line in file.readlines()]


    # output_1
    id_lst = []
    jfleg_lst_output = []
    counter = 1  # 初始化计数器
    for item in tqdm(jfleg_lst):
        id_lst.append(f"a_{counter:04d}")
        corrected_string = cor.correct_text(item)
        jfleg_lst_output.append(corrected_string)

        counter += 1

    data = {
        "id":id_lst,
        "text":jfleg_lst_output
    }
    df = pd.DataFrame(data)
    df.to_excel(output_file_path_1, index=False)

    # output_2
    id_lst = []
    jfleg_lst_output = []
    counter = 1  # 初始化计数器
    for item in tqdm(jfleg_lst):
        id_lst.append(f"a_{counter:04d}")
        corrected_string = cor.correct_text(item)
        jfleg_lst_output.append(corrected_string)

        counter += 1

    data = {
        "id":id_lst,
        "text":jfleg_lst_output
    }
    df = pd.DataFrame(data)
    df.to_excel(output_file_path_2, index=False)

    # output_3
    id_lst = []
    jfleg_lst_output = []
    counter = 1  # 初始化计数器
    for item in tqdm(jfleg_lst):

        ori_text = item
        cor_text = cor.correct_text(item)
        if calculate_semantic_similarity(ori_text, cor_text) < score:
            id_lst.append(f"a_{counter:04d}")
            jfleg_lst_output.append(ori_text)
        else:
            id_lst.append(f"b_{counter:04d}")
            jfleg_lst_output.append(cor_text)

        counter += 1

    data = {
        "id":id_lst,
        "text":jfleg_lst_output
    }
    df = pd.DataFrame(data)
    df.to_excel(output_file_path_3, index=False)

  0%|          | 0/1601 [00:00<?, ?it/s]

/home/liujunhui/miniconda3/envs/semantic_uncertainty/lib/python3.11/site-packages/text_utils/modules/grouping.py:36: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(enabled=False):


  0%|          | 0/1601 [00:00<?, ?it/s]

  0%|          | 0/1601 [00:00<?, ?it/s]

  0%|          | 0/1601 [00:00<?, ?it/s]

  0%|          | 0/1601 [00:00<?, ?it/s]

  0%|          | 0/1601 [00:00<?, ?it/s]